In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wid
from collections import namedtuple

c=3*10**8
e0=8.854*10**-12

class Param(namedtuple('Param', "default, range, text, tipo")):
    def make_widget(self):
        
        #Crea un widget deslizante por cada parámetro
        style={'description_width': 'initial'}
        if self.tipo==2:
        
            wtype = wid.Checkbox
            return wtype(description=self.text)
        else:
            wtype = wid.FloatSlider
            return wtype(value=self.default, min=self.range[0], max=self.range[1], 
                         step=self.range[2], continuous_update=True, description=self.text, style=style)
       
    
class Model:
    def interaction(self):
        widgetlist = {name:p.make_widget() for name, p in self.params.items()}
        wid.interact(self.run, **widgetlist)

class DobleRendija(Model):

    cons=50*10**-2*c/(5*10**14*5*10**-3)
    
    params = dict(
        fr=Param(5,(1, 9, 0.5),"Frecuencia (Hz 10*14)",10), #frecuencia
        D=Param(5,(0, 10, 0.01),"Distancia entre rendijas (mm)",1), #Distancia entre rendijas
        y=Param(50,(0, 100, 1),"Distancia a la pantalla (cm)",1), #Distancia a la pantalla
        b=Param(0.1,(0, 1, 0.1),"Anchura rendijas (mm)",1), #Anchura rendijas
        a=Param(6*cons,(cons,75*cons,cons/2),"Limite pantalla (m)",1), #limite pantalla
        A0=Param(1,(0, 100, 1),"Doble Rendija",2),)
        
       
    def run(self,fr,D,y,b,A0,a):
        w=2*np.pi*fr*10**14
        k=w/c
        D=D*10**-3
        b=b*10**-3
        y=y*10**-2
        
        z=np.linspace(-a,a,100000)
    
        r1=np.sqrt(((z-(D/2))**2)+(y**2))
        r2=np.sqrt(((z+(D/2))**2)+(y**2))
        A1=(np.exp(1j*k*r1)/np.sqrt(k*r1))

        A2=A0*(np.exp(1j*k*r2)/np.sqrt(k*r2))
        I=e0*w**2*c*np.abs(np.real((A1+A2)*(np.exp(-1j*np.pi/4)*k*b)/np.sqrt(np.pi*2))**2)/2
        plt.plot(z,I)
        plt.xlabel('Eje z en la pantalla (m)')
        plt.ylabel('Intensidad onda EM (unidad arbitraria)')

        plt.show()

DobleRendija().interaction()

interactive(children=(FloatSlider(value=5.0, description='Frecuencia (Hz 10*14)', max=9.0, min=1.0, step=0.5, …